In [1]:
import pandas as pd

df = pd.read_csv('/content/univ-length-over10.short-col-names.csv')

#df=df.drop(columns = ['date'])
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=2)
#df = imputer.fit_transform(df)

from sklearn.preprocessing import LabelEncoder
le1= LabelEncoder()
le2= LabelEncoder()
le3= LabelEncoder()
df['predWillFollowIncreaseCapacity']=le1.fit_transform(df['predWillFollowIncreaseCapacity'])
df['Ft-ULen-Over10-didFollowIncreaseCapacity']=le2.fit_transform(df['Ft-ULen-Over10-didFollowIncreaseCapacity'])

'''
df=df.dropna()
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le1 = LabelEncoder()
le2 = LabelEncoder()
'''


#df['icon']= le.fit_transform(df['icon'])

y = df['Ft-ULen-Over10-didFollowIncreaseCapacity'].values
df=df.drop(columns = ['Ft-ULen-Over10-didFollowIncreaseCapacity'])
X = df.values


In [ ]:
from collections import Counter
Counter(y)

Counter({0: 359, 1: 235})

In [2]:
X=X[1:]
y=y[1:]

In [3]:
for i in range(len(y)):
    if y[i]==2:
        y[i]=1

In [ ]:
y

In [4]:

y=y.reshape(-1, 1)


import numpy as np
X=np.concatenate((X[1:], y[0:-1]), axis=1)

In [ ]:
pip install keras-self-attention

  Created wheel for keras-self-attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18912 sha256=87267e5d0e697dca9a1aedad98cd877492fb44962202b633ea0393fae99b28cf
  Stored in directory: /root/.cache/pip/wheels/95/b1/a8/5ee00cc137940b2f6fa198212e8f45d813d0e0d9c3a04035a3
Successfully built keras-self-attention


In [5]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X= sc.fit_transform(X)

In [ ]:
pip install pyrcn

     |████████████████████████████████| 81 kB 6.2 MB/s 


In [6]:
from sklearn.model_selection import train_test_split
x_train1, x_test1, y_train1, y_test1 = train_test_split(X, y[1:], test_size=0.2, shuffle=False, random_state=42)


In [ ]:
x_train1.shape

(473, 20)

In [7]:
from pyrcn.base.blocks import InputToNode
from sklearn . datasets import make_blobs
# Generate a toy dataset

In [8]:
input_to_node = InputToNode (100, input_activation='relu',input_scaling =1.0 )


x_train= input_to_node.fit_transform (x_train1)
x_test= input_to_node.transform (x_test1)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:77: FutureWarning: Pass hidden_layer_size=100 as keyword args. From version 1.1 (renaming of 0.26) passing these as positional arguments will result in an error
  FutureWarning,


In [9]:
from pyrcn.base.blocks import NodeToNode
node_to_node = NodeToNode (100, reservoir_activation='relu', spectral_radius =1.0 , leakage =0.5 ,bidirectional = False )
x_train=node_to_node . fit_transform(x_train)
x_test= node_to_node.transform (x_test)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:77: FutureWarning: Pass hidden_layer_size=100 as keyword args. From version 1.1 (renaming of 0.26) passing these as positional arguments will result in an error
  FutureWarning,


In [10]:
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

x_train1 = x_train1.reshape((x_train1.shape[0], x_train1.shape[1], 1))
x_test1 = x_test1.reshape((x_test1.shape[0], x_test1.shape[1], 1))

In [11]:
from tensorflow import keras
from tensorflow.keras import layers

In [12]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [13]:
from keras_self_attention import SeqSelfAttention
def build_model(
    input_shapey,
    input_shapez,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputsy = keras.Input(shape=input_shapey)
    inputsz = keras.Input(shape=input_shapez)
    y = inputsy
    z= inputsz

    #z=layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(z)

    #y=layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(y)

    z=SeqSelfAttention()(z)

    #y=SeqSelfAttention()(y)



    x=layers.Concatenate(axis=1)([y, z])
    #x=layers.Add()([inputsy, inputsz])

    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(2, activation="softmax")(x)
    #outputs = layers.Dense(1)(x)
    return keras.Model([inputsy,inputsz], outputs)

In [14]:
input_shapey = x_train1.shape[1:]
input_shapez = x_train.shape[1:]

model = build_model(
    input_shapey,
    input_shapez,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.7,
    dropout=0.75,
)
'''
model.compile(
    loss="mean_absolute_error",
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=["mean_absolute_error"],
)
'''
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-2),
    metrics=["accuracy"],
)

model.summary()





Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 100, 1)]     0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 20, 1)]      0           []                               
                                                                                                  
 seq_self_attention (SeqSelfAtt  (None, 100, 1)      129         ['input_2[0][0]']                
 ention)                                                                                          
                                                                                                  
 concatenate (Concatenate)      (None, 120, 1)       0           ['input_1[0][0]',            

In [15]:

callbacks = [keras.callbacks.EarlyStopping(patience=80, restore_best_weights=True)]

history=model.fit(
    [x_train1,x_train],
    y_train1,
    validation_data=([x_test1, x_test], y_test1),
    epochs=1000,
    batch_size=128,
    callbacks=callbacks,
)

Epoch 1/1000
4/4 [==============================] - 6s 307ms/step - loss: 1.6532 - accuracy: 0.5391 - val_loss: 0.5788 - val_accuracy: 0.6891
Epoch 2/1000
4/4 [==============================] - 0s 103ms/step - loss: 0.9470 - accuracy: 0.5941 - val_loss: 0.6247 - val_accuracy: 0.6639
Epoch 3/1000
4/4 [==============================] - 0s 103ms/step - loss: 0.7489 - accuracy: 0.6744 - val_loss: 0.7499 - val_accuracy: 0.5714
Epoch 4/1000
4/4 [==============================] - 0s 111ms/step - loss: 0.6842 - accuracy: 0.6681 - val_loss: 0.5657 - val_accuracy: 0.6891
Epoch 5/1000
4/4 [==============================] - 0s 103ms/step - loss: 0.6009 - accuracy: 0.7146 - val_loss: 0.5792 - val_accuracy: 0.7479
Epoch 6/1000
4/4 [==============================] - 0s 105ms/step - loss: 0.5816 - accuracy: 0.7230 - val_loss: 0.5736 - val_accuracy: 0.7059
Epoch 7/1000
4/4 [==============================] - 0s 107ms/step - loss: 0.5784 - accuracy: 0.6956 - val_loss: 0.5463 - val_accuracy: 0.7311
Epoch 

In [16]:
#testing prediction

#testing prediction
ypred1=model.predict([x_test1, x_test])

ypred=[]
#testing prediction
for i in ypred1:
    if i[0]>i[1]:
        ypred.append(0)
    else:
        ypred.append(1)
from sklearn.metrics import accuracy_score, f1_score,  jaccard_score, roc_auc_score, precision_score, recall_score
print(accuracy_score(ypred, y_test1))



print('f1_score', f1_score(ypred, y_test1))



print('jaccard_score', jaccard_score(ypred, y_test1))
print('roc_auc_score', roc_auc_score(ypred, y_test1))
print('precision_score', precision_score(ypred, y_test1))

print('recall_score', recall_score(ypred, y_test1))


0.8403361344537815
f1_score 0.7710843373493975
jaccard_score 0.6274509803921569
roc_auc_score 0.8540829986613119
precision_score 0.6808510638297872
recall_score 0.8888888888888888


In [17]:
#training prediction
xpred1=model.predict([x_train1, x_train])
xpred=[]

for i in xpred1:
    if i[0]>i[1]:
        xpred.append(0)
    else:
        xpred.append(1)
from sklearn.metrics import accuracy_score, f1_score,  jaccard_score, roc_auc_score, precision_score, recall_score
print('accuracy', accuracy_score(xpred, y_train1))



print('f1_score', f1_score(xpred, y_train1))



print('jaccard_score', jaccard_score(xpred, y_train1))
print('roc_auc_score', roc_auc_score(xpred, y_train1))
print('precision_score', precision_score(xpred, y_train1))

print('recall_score', recall_score(xpred, y_train1))


accuracy 0.8942917547568711
f1_score 0.8511904761904763
jaccard_score 0.7409326424870466
roc_auc_score 0.913877338877339
precision_score 0.7606382978723404
recall_score 0.9662162162162162
